In [1]:
import os

# カレントディレクトリをmagentaに
os.chdir(r"C:\Users\arkw\GitHub\magenta")
print(os.getcwd())

C:\Users\arkw\GitHub\magenta


In [2]:
def calculate_c_diatonic(notes):
    """Cのダイアトニック・スケールに該当する音符の割合を計算
    引数:
        notes: pretty_midi.Noteオブジェクトのリスト
    戻り値:
        Cのダイアトニック・スケールに該当する音符の割合
    """
    c_diatonic_scale = {0, 2, 4, 5, 7, 9, 11}
    diatonic_notes = [(note.pitch % 12) in c_diatonic_scale for note in notes]
    return sum(diatonic_notes) / len(notes) if notes else 0

def calculate_note_density(notes, tempo):
    """ノート密度の計算"""
    second_per_beat = 60 / tempo # 一拍あたりの秒数
    note_steps = notes[-1].end / second_per_beat * 4 # 16分音符のステップ数に変換
    return len(notes) / note_steps if note_steps else 0

def calculate_average_interval(notes):
    """平均音程間隔の計算"""
    intervals = [abs(notes[i].pitch - notes[i-1].pitch) for i in range(1, len(notes))]
    return sum(intervals) / len(intervals) if intervals else 0

def calculate_syncopation(notes, tempo, syncopation_type):
    """全ての音符に対するシンコペーション音符の割合を計算する
    引数:
        notes: pretty_midi.Noteオブジェクトのリスト
        tempo: テンポ。qpmと同義
        syncopation_type: 8thか16th
    戻り値:
        シンコペーション音符の割合
    """
    syncopated_count = 0 # シンコペーション音符の数
    prev_note_start = 0 # 一つ前の音符の開始時間
    second_per_beat = 60 / tempo # 一拍あたりの秒数
    if syncopation_type == "8th":
        value_for_evaluation = 0.5
    elif syncopation_type == "16th":
        value_for_evaluation = 0.25

    for note in notes:
        is_position = (note.start / second_per_beat) % (second_per_beat * 2) == value_for_evaluation
        interval = (note.start - prev_note_start) / second_per_beat
        if(is_position and interval > value_for_evaluation):
            syncopated_count += 1
        prev_note_start = note.start
    return (syncopated_count / len(notes)) if notes else 0

In [3]:
import pretty_midi

# 例外オブジェクトを作るためのクラスを定義
# 読み込んだMIDIファイルが条件に合わない場合に
# このクラスによって定義される例外が投げられる
class UnsupportedMidiFileException(Exception):
  "Unsupported MIDI File"

def analyze_midi(file_path):
    midi_data = pretty_midi.PrettyMIDI(file_path)
    _, tempi = midi_data.get_tempo_changes()
    tempo = int(tempi[0])
    time_signature = midi_data.time_signature_changes[0]
    if midi_data.instruments:
      instrument = midi_data.instruments[0]

      # 属性計算
      c_diatonic = calculate_c_diatonic(instrument.notes) # Cのダイアトニック・スケールに該当する音符の割合
      note_num = len(instrument.notes) # ノートの数
      note_density = calculate_note_density(instrument.notes, tempo) # ノート密度
      average_interval = calculate_average_interval(instrument.notes) # 平均音程間隔
      syncopation_16th = calculate_syncopation(instrument.notes, tempo, "16th") # 16分シンコペーション音符の割合
      syncopation_8th = calculate_syncopation(instrument.notes, tempo, "8th") # 8分シンコペーションノートの割合

      return (file_path, tempo, time_signature, 
              c_diatonic, note_num, note_density, average_interval, syncopation_16th, syncopation_8th)
    else:
       # 空の楽譜の場合はこう
       return (file_path, tempo, time_signature,
               0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [4]:
import glob
import os

midi_path = r"C:\Users\arkw\GitHub\magenta\tmp\music_vae\generated\0704"
files = glob.glob(os.path.join(midi_path, '*.mid'))

In [29]:
import csv

def write_to_csv(results, csv_path):
    # ディレクトリが存在しない場合は作成する
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
     # CSVファイルに結果を保存
    with open(csv_path, 'w', newline='') as csvfile:  # 書き込みモード
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(results)  # 結果を1行に書き込む

In [5]:
import pandas as pd

def write_to_csv(results, csv_path):
    # DataFrameを作成してCSVファイルに結果を保存
    df = pd.DataFrame(results, columns=['file_path', 'tempo', 'time_signature', 'c_diatonic', 'note_num', 'note_density', 'average_interval', 'syncopation_16th', 'syncopation_8th'])
    df.to_csv(csv_path, index=False)

In [6]:
import datetime

results = []
for file in files:
    results.append(analyze_midi(file))

now = datetime.datetime.now()
filename = 'tmp/result_' + now.strftime('%Y%m%d_%H%M%S') + '.csv'
write_to_csv(results, filename)

指定された条件を満たすMIDIファイルを絞り込み、別の場所に保存する

In [46]:
import pandas as pd
import shutil
import os

# CSVファイルの読み込み
csv_file_path = 'tmp/result_20240529_150740.csv'
df = pd.read_csv(csv_file_path)

# 条件を満たす行をフィルタリング
filtered_df = df[(df['note_num'] >= 4) & (df['average_interval'] < 8)]

# 保存先ディレクトリの指定
destination_dir = r'tmp\music_vae\generated\adjective01\filtered'
os.makedirs(destination_dir, exist_ok=True)

# 条件を満たすMIDIファイルをコピー
for file_path in filtered_df['file_path']:
    shutil.copy(file_path, destination_dir)

print(f"Filtered MIDI files have been saved to {destination_dir}")

Filtered MIDI files have been saved to tmp\music_vae\generated\adjective01\filtered


In [13]:
import pandas as pd
import numpy as np

# 取り出す属性はこれ！
attribute = 'syncopation_8th'

csv_path = "tmp/result_20240708_145215.csv"
df = pd.read_csv(csv_path)
# filtered_csv_path = "tmp/midi16bar_result100.csv"
# df = pd.read_csv(filtered_csv_path)

# c_diatonicの上位25%に属するfile_pathの一覧を配列で取得
top_25_percent = df[df[attribute] >= df[attribute].quantile(0.75)]['file_path'].tolist()

# c_diatonicの下位25%に属するfile_pathの一覧を配列で取得
bottom_25_percent = df[df[attribute] <= df[attribute].quantile(0.25)]['file_path'].tolist()

# dir = r"tmp\music_vae_16bar\generated\iec0007\vector\gen_001\\"

# file_path25個それぞれの末尾に.npyをつけて、np.loadする
# top_npy_files = [path + '.npy' for path in top_25_percent]
top_npy_files = [path.replace(".mid", ".npy") for path in top_25_percent]
top_npy_arrays = [np.load(npy_file) for npy_file in top_npy_files]

# bottom_npy_files = [path + '.npy' for path in bottom_25_percent]
bottom_npy_files = [path.replace(".mid", ".npy") for path in bottom_25_percent]
bottom_npy_arrays = [np.load(npy_file) for npy_file in bottom_npy_files]

# 属性ベクトルを求める
attribute_npy = np.mean(top_npy_arrays, axis=0) - np.mean(bottom_npy_arrays, axis=0)

print(attribute_npy)
np.save(attribute, attribute_npy)



[-6.57945350e-02  2.44449303e-02  7.53115341e-02  2.03467794e-02
 -2.30756737e-02  5.92178293e-02  3.58050987e-02 -2.12353952e-02
 -1.72579959e-02 -2.52422184e-01  8.57003927e-02  9.27222371e-02
  6.29353747e-02 -7.28392825e-02  3.68721068e-01 -1.77453458e-01
  1.05784096e-01 -5.22302538e-02 -1.82669729e-01 -6.55619949e-02
  6.53382316e-02 -1.75451994e-01 -7.53328204e-03  7.51072466e-02
 -1.35784969e-02 -4.61742021e-02 -1.13153458e-02  4.10844758e-02
 -1.12348914e-01  1.49679333e-01  7.59288296e-02  1.55146252e-02
 -5.59456125e-02 -1.54384002e-01  1.26174927e-01  2.26169955e-02
  3.21011506e-02  2.99045350e-02 -2.12514162e-01 -1.19904473e-01
 -3.13224271e-02 -7.80632300e-03 -1.51096284e-03  1.90484852e-01
 -5.08893952e-02 -2.03543659e-02  1.59708373e-02  1.80093404e-02
  2.20194504e-01  8.68370980e-02  2.08973382e-02 -1.66575149e-01
  8.30930173e-02 -1.55522972e-01 -2.00085670e-01  1.22280695e-01
 -3.08950376e-02 -6.34417310e-03 -1.06051981e-01 -3.50334607e-02
 -7.29732215e-03  1.62791

: 

In [ ]:
import pandas as pd
import numpy as np

# 取り出す属性はこれ！
attribute = 'c_diatonic'

csv_path = "magenta/tmp/result_20240708_145215.csv"
df = pd.read_csv(csv_path)
# filtered_csv_path = "tmp/midi16bar_result100.csv"
# df = pd.read_csv(filtered_csv_path)

# c_diatonicの上位25%に属するfile_pathの一覧を配列で取得
top_25_percent = df[df[attribute] >= df[attribute].quantile(0.75)]['file_path'].tolist()

# c_diatonicの下位25%に属するfile_pathの一覧を配列で取得
bottom_25_percent = df[df[attribute] <= df[attribute].quantile(0.25)]['file_path'].tolist()

dir = r"tmp\music_vae_16bar\generated\iec0007\vector\gen_001\\"

# file_path25個それぞれの末尾に.npyをつけて、np.loadする
# top_npy_files = [path + '.npy' for path in top_25_percent]
top_npy_files = [dir + path for path in top_25_percent]
top_npy_arrays = [np.load(npy_file) for npy_file in top_npy_files]

# bottom_npy_files = [path + '.npy' for path in bottom_25_percent]
bottom_npy_files = [dir + path for path in bottom_25_percent]
bottom_npy_arrays = [np.load(npy_file) for npy_file in bottom_npy_files]

# 属性ベクトルを求める
attribute_npy = np.mean(top_npy_arrays, axis=0) - np.mean(bottom_npy_arrays, axis=0)

print(attribute_npy)
np.save(attribute, attribute_npy)



[[-1.78200126e-01  8.17373097e-02  1.06614143e-01  7.47459307e-02
   1.28372476e-01 -2.25054752e-02 -1.58409342e-01  2.17874870e-01
   3.09341729e-01 -7.68767297e-03 -1.06021449e-01  7.87273496e-02
   1.83909416e-01 -1.65761843e-01 -1.85423315e-01  2.27942318e-03
  -2.37687200e-01  1.73326641e-01  2.11889386e-01  1.16456077e-02
   1.48005694e-01  1.72426790e-01 -2.46435806e-01 -6.78439736e-02
   1.84463501e-01 -4.14486900e-02  1.27142265e-01  5.65832034e-02
   5.72618723e-01 -1.97568893e-01 -3.74879122e-01  7.31136575e-02
   3.65268588e-01 -3.79895985e-01 -3.72900397e-01  1.45070374e-01
  -2.82924235e-01  2.74517447e-01 -1.08691975e-01 -3.29487622e-01
  -1.44560963e-01  1.14685223e-02 -1.03256758e-02  1.60682350e-02
   8.23171437e-02 -1.47445321e-01  3.43763560e-01  1.20125808e-01
  -5.91517389e-02  8.61423910e-02  3.27624902e-02 -1.67826399e-01
   8.10731649e-02 -4.84586060e-02  8.88331980e-03 -7.02095106e-02
   3.25084805e-01 -2.58288294e-01  2.76272357e-01  7.37924278e-02
   2.56150

In [11]:
import pandas as pd
import numpy as np

filtered_csv_path = "tmp/filtered20240610.csv"
df = pd.read_csv(filtered_csv_path)

# c_diatonicの上位25%に属するfile_pathの一覧を配列で取得
top_25_percent = df[df['c_diatonic'] >= df['c_diatonic'].quantile(0.75)]['file_path'].tolist()

# c_diatonicの下位25%に属するfile_pathの一覧を配列で取得
bottom_25_percent = df[df['c_diatonic'] <= df['c_diatonic'].quantile(0.25)]['file_path'].tolist()

# file_path25個それぞれの末尾に.npyをつけて、np.loadする
top_npy_files = [path + '.npy' for path in top_25_percent]
top_npy_arrays = [np.load(npy_file) for npy_file in top_npy_files]

bottom_npy_files = [path + '.npy' for path in bottom_25_percent]
bottom_npy_arrays = [np.load(npy_file) for npy_file in bottom_npy_files]

# 上位25%平均を求める
average_npy = np.mean(top_npy_arrays, axis=0)

print(average_npy)


[-1.77503332e-01 -6.97756782e-02  1.14540532e-01 -3.48111056e-02
  2.03189943e-02 -1.17891952e-01 -9.36871544e-02  2.19116837e-01
  7.98813999e-02  3.80063444e-01  1.32374719e-01  2.28011608e-01
 -1.59327179e-01 -7.66465366e-01  2.00941842e-02  1.52916208e-01
 -1.44675344e-01 -2.63341874e-01  4.49163318e-01 -6.65007606e-02
  3.69625562e-03  2.42318675e-01  2.81475991e-01  3.94095220e-02
  1.79621145e-01  1.74960911e-01  3.42363089e-01  2.03059893e-02
  7.86448419e-02 -4.12395060e-01 -6.59564137e-02 -1.05539776e-01
  2.09795043e-01 -9.97529700e-02  7.42982626e-02  8.98024812e-02
  1.10022470e-01 -1.36720270e-01 -9.98988748e-02 -6.85084462e-02
  2.05507185e-02 -7.27124095e-01  2.93461084e-01 -3.49983037e-01
 -1.74424872e-01 -3.09932902e-02  1.19998731e-01  2.84753770e-01
  7.35256597e-02  1.43863976e-01  9.19394046e-02 -1.45569041e-01
 -1.11884005e-01 -4.41754237e-03 -1.71349436e-01 -5.20440191e-02
  5.71642280e-01 -2.15549529e-01 -5.03585190e-02 -6.98920861e-02
 -2.30890065e-02  8.92823